In [1]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
import plotly.express as px
from dash.exceptions import PreventUpdate


# Import CRUD AnimalShelter module (replace with your module name)
from AnimalShelter import CrudAnimalShelter

# Set up username and password for database access
username = "aacuser"
password = "Pokemon845!"

# Instantiate the CRUD AnimalShelter database object
db = CrudAnimalShelter(username, password)

# Fetch data from the database and create a DataFrame
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

# Create the main app object
app = JupyterDash(__name__)

# Define the layout of the app using HTML and Dash components
# Dashboard Layout / View
app.layout = html.Div([
    html.Img(src="http://localhost:6852/view/GraziosoSalvareLogo.png", style={'width': '200px', 'float': 'right', 'padding': '10px'}),
    html.Center(html.B(html.H1('Joseph Valle'))),
    html.Hr(),
    html.Div(
        className='buttonRow', 
        style={'display' : 'flex'},
        children=[
            html.Button(id='submit-button-one', n_clicks=0, children='Cats'),
            html.Button(id='submit-button-two', n_clicks=0, children='Dogs')
        ]
    ),
    dcc.Dropdown(
        id='animal-type-dropdown',
        options=[
            {'label' : 'All', 'value' : 'all'},
            {'label' : 'Cats', 'value' : 'Cat'},
            {'label' : 'Dogs', 'value' : 'Dog'}
        ],
        value='all',        
        clearable=False
    ),
    dcc.RangeSlider(
        id='age-range-slider',
        min = 0,
        max = 20,
        step = 1,
        marks = {i: str(i) for i in range(0, 21)},
        value = [0, 20]
    ),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable=False,
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
    html.Div(id='geolocation-chart', style={'width': '100%', 'height': '300px'}),
    dcc.Graph(id='bar-chart')
])

# Callback to update data table based on filters
@app.callback(
    Output('datatable-id', "data"),
    [Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two', 'n_clicks'),
     Input('animal-type-dropdown', 'value'),
     Input('age-range-slider', 'value')]
)
def update_data_table(button1, button2, selected_animal_type, age_range):
    min_age, max_age = age_range
    if selected_animal_type == 'all':
        query = {"age_upon_outcome_years": {"$gte": min_age, "$lte": max_age}}
    else:
        query = {"animal_type": selected_animal_type, "age_upon_outcome_years": {"$gte": min_age, "$lte": max_age}}
    df_filtered = pd.DataFrame.from_records(db.read(query))
    df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records')

# Callback to update geolocation chart based on selected row
@app.callback(Output('geolocation-chart', 'children'),
              [Input('datatable-id', 'selected_rows')])
def update_geolocation_chart(index):
    if index is None:
        raise PreventUpdate
    row = index[0]
    lat = df.iloc[row, 13]
    lon = df.iloc[row, 14]
    return [
        dl.Map(style={'width': '100%', 'height': '300px'},
               center=[lat, lon], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[lat, lon],
                             children=[
                                 dl.Tooltip(df.iloc[row, 4]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(df.iloc[row, 9])
                                 ])
                             ])
               ])
    ]

# Callback to update bar chart based on selected row
@app.callback(Output('bar-chart', 'figure'),
              [Input('datatable-id', 'selected_rows')])
def update_bar_chart(index):
    if index is None:
        raise PreventUpdate
    row = index[0]
    selected_breed = df.iloc[row, 4]
    breed_counts = df['breed'].value_counts()
    fig = px.bar(breed_counts, x=breed_counts.index, y=breed_counts.values,
                 title=f"Breed Distribution for {selected_breed}",
                 labels={'x': 'Breed', 'y': 'Count'})
    return fig

# Run the app in debug mode
app.run_server(debug=True)



Dash app running on http://127.0.0.1:10852/
